# Imports

In [1]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pandas as pd

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
34,application_1643288788512_0039,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


# Leemos el fichero de usuarios

In [2]:
df1 = spark.read.format("csv").load("/data/userid-profile.tsv", sep='\t', header=True, inferSchema=True)

# Pasamos el DF a Pandas para operar con el

In [3]:
pandasDFusuarios = df1.toPandas()

# Obtenemos los que no tienen id, pais o fecha de registro

In [4]:
df_usuarios_sin_datos = pandasDFusuarios[pandasDFusuarios['id'].isnull() | pandasDFusuarios['country'].isnull() | pandasDFusuarios['registered'].isnull() ]

# Nos quedamos con los que tienen todos los datos

In [5]:
df_usuarios_completos = pandasDFusuarios
df_usuarios_completos.drop(df_usuarios_sin_datos.index, inplace = True)

# El Panda lo paso a DF para escribirlo en fichero y tabla Hive

In [6]:
sparkDF=spark.createDataFrame(df_usuarios_completos)

# Escribo en fichero el resultado

In [ ]:
sparkDF.write.format("csv").save('wasb://containergrupo1clusterspark@grupo1clusterhdistorage.blob.core.windows.net/data/userid-profile-clear.csv',header = 'true')

# Creo la estructura de la tabla HIVE

In [19]:
%%sql -q
CREATE TABLE IF NOT EXISTS useridProfile ( 
                    id string, 
                    gender string, 
                    age int,
                    country string,
                    registered string)

# Guardo los datos en la tabla

In [8]:
dfw = DataFrameWriter(sparkDF)
dfw.insertInto('useridProfile', overwrite=True)

# Compruebo para ver que hay datos en la tabla

In [9]:
datosdf = spark.sql("""
SELECT * 
FROM useridProfile 
""")
datosdf.show()

+-----------+------+---+------------------+------------+
|         id|gender|age|           country|  registered|
+-----------+------+---+------------------+------------+
|user_000001|     m|  0|             Japan|Aug 13, 2006|
|user_000002|     f|  0|              Peru|Feb 24, 2006|
|user_000003|     m| 22|     United States|Oct 30, 2005|
|user_000005|     m|  0|          Bulgaria|Jun 29, 2006|
|user_000006|  null| 24|Russian Federation|May 18, 2006|
|user_000007|     f|  0|     United States|Jan 22, 2006|
|user_000008|     m| 23|          Slovakia|Sep 28, 2006|
|user_000009|     f| 19|     United States|Jan 13, 2007|
|user_000010|     m| 19|            Poland| May 4, 2006|
|user_000011|     m| 21|           Finland| Sep 8, 2005|
|user_000012|     f| 28|     United States|Mar 30, 2005|
|user_000013|     f| 25|           Romania|Sep 25, 2006|
|user_000015|  null| 21|     Cote D'Ivoire| Oct 3, 2006|
|user_000016|     m|  0|    United Kingdom| Aug 5, 2005|
|user_000017|     m| 22|       